In [1]:
%run notebook.config.ipy


Attaching package: ‘fastcluster’

The following object is masked from ‘package:stats’:

    hclust




Attaching package: ‘reshape2’

The following objects are masked from ‘package:reshape’:

    colsplit, melt, recast




Attaching package: ‘gplots’

The following object is masked from ‘package:stats’:

    lowess



Loading required package: DBI


set database db: ../csvdb
set database anndb: /gfs/mirror/annotations/mm10_ensembl78/csvdb
set database ipydb: csvdb


*Unmelt the count data to a new table*

In [4]:
statement = '''select * from htseq_counts'''

counts = DB.fetch_DataFrame(statement, db)
count_table = counts.pivot(columns="track", index="gene_id", values="counts")

print count_table.shape

DB.write_DataFrame(count_table,"count_table",ipydb)
#count_table.to_csv("count_table.txt",sep="\t")


(22116, 99)


*Select the cells based on the QC metrics*

In [17]:
# e.g. select cells expressing more than 3000 genes

statement = '''select cell 
               from qc_summary q
               where q.no_genes > 3000'''

good_cells = DB.fetch_DataFrame(statement, db)["cell"].values

print len(good_cells)

59


*Prepare table of selected cells*

In [19]:
cell_stat = '"' + '", "'.join(good_cells) + '"'

# fetch a pandas dataframe containing the "good" cells
statement = '''select gene_id, %(cell_stat)s
               from count_table''' % locals()

count_table_filtered = DB.fetch_DataFrame(statement, ipydb)
print count_table_filtered.shape

# write a new frame containing the filtered data
DB.write_DataFrame(count_table_filtered, "count_table_filtered", ipydb)

(22116, 60)
